In [ ]:
# import
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
from datetime import datetime
import pandas as pd
import time
import random
import os

# 주피터 파일이랑 같은 폴더에 chromedriver 넣어야합니다.
driver = webdriver.Chrome('chromedriver.exe')
soup = bs(driver.page_source, 'html.parser')
driver.implicitly_wait(3)

# 이렇게 입력할 수 도 있고, 텍스트파일로 만들어둬서 입출력 자동화해도되구~
swarm_ID=input("ID를 입력해주세요 : ")
swarm_PW=input("PW를 입력해주세요 : ")

# swarm login url 접속
driver.get('https://ko.swarmapp.com/login')

# ID, PW 텍스트박스에 정보 입력
driver.find_element_by_name('emailOrPhone').send_keys(swarm_ID)
driver.find_element_by_name('password').send_keys(swarm_PW)

#로그인 버튼 클릭
driver.find_elements_by_css_selector("p.loginOrSignup > input")[0].click()

In [ ]:
# 경로 목록으로 들어가는 작업.
# 이거 안하면 엉뚱한거 뽑힘
driver.find_elements_by_css_selector("div.user-arrow > img")[0].click()
driver.find_elements_by_css_selector("li > a.userHistoryLink")[0].click()

In [ ]:
from selenium.webdriver.common.keys import Keys
element=driver.find_element_by_tag_name('html')
now_len = 0

while 1:
    # 50개씩만 보이고, 스크롤 내려가면 추가적으로 보이는 구조
    # 스크롤을 끝까지 내려서 모든 데이터가 보여야해서 만든 코드
    # 근데 이부분이 살짝 엉킬수도있음..
    # 250 좀 조정하면서 해보고
    # 끝까지 안내려갔는데 그냥 finish되면 그냥 손으로 스크롤 내리셈
    for j in range(250):
        element.send_keys(Keys.ARROW_DOWN)    
    #time.sleep(3)
    locate_datas = driver.find_elements_by_css_selector('div.activity')
    if now_len == len(locate_datas):
        print('finish')
        break
    now_len = len(locate_datas)
    print('now_len : ' + str(now_len))

In [ ]:
soup = bs(driver.page_source, 'html.parser')
# n일전을 계산하기 위한, 코드 수행 순간 날짜
now = datetime.now()
print(str(now.year)+"년 "+str(now.month)+"월 "+str(now.day)+"일")

# 실질적인 수집부
for j in range(now_len):
    print(str(j+1)+"..", end=" : ")
    print(soup.select('span.activity-checkinInfo-venue > a')[j].get_text(), end=", ")
    print(soup.select('p > span.global')[j].get_text(), end=", ")
    print(soup.select('span.timestamp.autoupdate.separateLink')[j].get_text(), end=", ")    
    print("")
print('finish..')